In [28]:
import numpy as np
import pandas as pd
import dask.bag as db
import json
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
tagged_documents = [TaggedDocument(words=preprocess_string(doc), tags=[i]) for i, doc in zip(df.index,df.abstract)]

model_d2v = Doc2Vec(vector_size=300, min_count=4,epochs=30,dm=1,window=5)
model_d2v.build_vocab(tagged_documents)

model_d2v.train(tagged_documents,
              total_examples=model_d2v.corpus_count,
              epochs=model_d2v.epochs)
document_embeddings=np.zeros((df.shape[0],300))

for i,value in enumerate(df.index.values):
    document_embeddings[i]=model_d2v.docvecs[value]
    
    
train,test = split_data(df)

X_train = np.array([model_d2v.docvecs[x] for x in train.index])
X_test = np.array([model_d2v.docvecs[x] for x in test.index])
y_train = train[categories_cols]
y_test = test[categories_cols]

In [4]:
model.docvecs[1]

/var/folders/v7/wxn3bxmn2018mjps58z5d3hw0000gn/T/ipykernel_41791/686413501.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs[1]


array([ 1.04582414e-01,  3.93723339e-01,  9.59099382e-02,  1.00177424e-02,
       -3.27561229e-01, -5.03894031e-01, -2.12235644e-01,  5.60929120e-01,
        4.50294882e-01,  4.73854840e-02, -6.09767511e-02, -2.26938054e-01,
       -4.58065152e-01,  5.96811712e-01, -3.25308502e-01, -2.07536399e-01,
        5.82209527e-01,  2.38266319e-01,  4.97709513e-01,  1.86664179e-01,
        8.55691135e-02, -3.69386971e-01,  4.69126970e-01,  4.30077106e-01,
       -1.14891753e-01, -7.22964332e-02,  2.02200428e-01,  4.19819176e-01,
       -3.25621553e-02, -5.94676733e-01,  2.51555741e-01,  3.88295770e-01,
       -1.55783683e-01, -6.65538728e-01, -3.17352623e-01, -3.24370980e-01,
        5.09900808e-01, -4.78036642e-01,  6.91409707e-02, -1.42265245e-01,
       -6.65231654e-03,  1.19720295e-01,  3.69992435e-01, -2.14161143e-01,
        3.95463467e-01, -1.28142849e-01, -1.81739539e-01,  1.38552710e-02,
       -5.44693232e-01, -1.21992575e-02, -4.51958537e-01, -2.92049408e-01,
       -4.71717656e-01,  

In [2]:
docs = db.read_text('./arxiv-metadata-oai-snapshot.json').map(json.loads)
trim = lambda x: {'id': x['id'],
                  'title': x['title'],
                  'category':x['categories'].split(' '),
                  'abstract':x['abstract']}

# filter for papers published on or after 2019-01-01
columns = ['id','category','abstract']
docs_df = (docs
             .filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2018)
             .map(trim)
             .compute())

# convert to pandas
docs_df = pd.DataFrame(docs_df)

In [22]:
X, y_array = for_train('coauthor', p=0.5, type='numpy')
print(X.shape)

delet some useless data: 100%|██████████| 25793/25793 [00:00<00:00, 117948.01it/s]


Number of instance with label :  7460
Number of instance without label(remain) :  7460


authors: 100%|██████████| 25793/25793 [00:03<00:00, 7775.49it/s]

(14920, 21146)


In [23]:
y = []
for i in y_array:
    if sum(i) > 0:
        y.append(1)
    else:
        y.append(0)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
# X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
# clf = svm.SVC().fit(X_train, y_train)

# clf.predict(X_train, y_train)
# clf.predict_proba(X[:2, :])

# clf.score(X_test, y_test)
clf.score(X_train, y_train)

0.9470788315326131

In [26]:
y_pred = clf.predict(X_test)

In [27]:
print(sum(y_test) / len(y_test))
print(sum(y_pred) / len(y_pred))

0.5040617384240454
0.5322908204711616


In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup, AutoModel, AutoTokenizer, BertConfig
import pytorch_lightning as pl
# from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from preprocessing import *

%matplotlib inline
%config InlineBackend.figure_format='retina'
RANDOM_SEED = 42
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 2e-5

# tokenizer = AutoTokenizer.from_pretrained('roberta-base')
BERT_MODEL_NAME = 'bert-base-cased'
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [5]:
import json

f_train = open("../../data/train.json", 'r')
train_data = json.load(f_train)

def get_data(data, key):
    di = find_discard_authors(train_data, p=0.20250)

    result = pd.DataFrame()

    X = []
    y = []

    for i in tqdm(range(len(data))):
        if i not in di:
            instance = data[i]
            abstract = instance['abstract']
            title = instance['title']
            authors = instance[key]

            # sentence
            sentence = ''
            for j in abstract:
                sentence += str(j) + ' '
            for j in title:
                sentence += str(j) + ' '
            X.append(sentence[:-1])

            # labels
            tmp = [0 for  _ in range(100)]
            for j in authors:
                if j < 100:
                    tmp[j] += 1
            y.append(tmp)
    y = np.array(y)
    result['text'] = X
    for i in range(100):
        # for j in range(len(X)):
        result[f'label{i}'] = y[:, i]

    return result

In [6]:
MAX_TOKEN_COUNT = 5000
LABEL_COLUMNS = [f'label{i}' for i in range(100)]
train_df = get_data(train_data, 'authors')

delet some useless data: 100%|██████████| 25793/25793 [00:00<00:00, 376100.08it/s]


Number of instance with label :  7460
Number of instance without label(remain) :  1894


100%|██████████| 25793/25793 [00:05<00:00, 4843.01it/s]
/var/folders/v7/wxn3bxmn2018mjps58z5d3hw0000gn/T/ipykernel_95963/618464660.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[f'label{i}'] = y[:, i]


In [7]:
test_size = 0.2
train_index = random.sample(list(train_df.index), int((1 - test_size) * len(train_df)))

train_set = train_df.loc[train_index, :]
test_set = train_df.drop(train_index, axis=0)

print("Train:")
print("     train_set : ", train_set.shape)

print("Test_Kaggle:")
print("     test_set  : ", test_set.shape)

Train:
     train_set :  (7483, 101)
Test_Kaggle:
     test_set  :  (1871, 101)


In [8]:
train_set.head(1)

,text,label0,label1,label2,label3,label4,label5,label6,label7,label8,...,label90,label91,label92,label93,label94,label95,label96,label97,label98,label99
4305,47 1548 2566 1525 2801 1529 2387 1896 1549 227...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
sample_instance = train_set['text'][0]

In [11]:
encoding = tokenizer.encode_plus(
    sample_instance,
    add_special_tokens=True,
    max_length=512,
    return_token_type_ids=False,
    padding="max_length",
    return_attention_mask=True,
    return_tensors='pt'   
)

In [13]:
# encoding['input_ids']

In [16]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'].squeeze())

['<s>',
 '24',
 '55',
 'Ġ18',
 '58',
 'Ġ2',
 '335',
 'Ġ15',
 '43',
 'Ġ1800',
 'Ġ1860',
 'Ġ2000',
 'Ġ28',
 '67',
 'Ġ15',
 '46',
 'Ġ18',
 '74',
 'Ġ20',
 '59',
 'Ġ15',
 '25',
 'Ġ25',
 '90',
 'Ġ4',
 '196',
 'Ġ12',
 'Ġ26',
 '34',
 'Ġ15',
 '43',
 'Ġ1800',
 'Ġ15',
 '86',
 'Ġ28',
 '66',
 'Ġ3',
 '595',
 'Ġ18',
 '66',
 'Ġ16',
 '70',
 'Ġ2000',
 'Ġ37',
 '43',
 'Ġ15',
 '42',
 'Ġ16',
 '50',
 'Ġ15',
 '27',
 'Ġ33',
 'Ġ4',
 '407',
 'Ġ15',
 '43',
 'Ġ15',
 '35',
 'Ġ1962',
 'Ġ1961',
 'Ġ15',
 '43',
 'Ġ33',
 'Ġ1700',
 'Ġ15',
 '43',
 'Ġ15',
 '35',
 'Ġ16',
 '47',
 'Ġ15',
 '46',
 'Ġ15',
 '80',
 'Ġ47',
 '20',
 'Ġ12',
 'Ġ17',
 '31',
 'Ġ4',
 '231',
 'Ġ26',
 '01',
 'Ġ15',
 '53',
 'Ġ17',
 '04',
 'Ġ16',
 '05',
 'Ġ24',
 '56',
 'Ġ15',
 '43',
 'Ġ3',
 '281',
 'Ġ15',
 '94',
 'Ġ4',
 '407',
 'Ġ2',
 '168',
 'Ġ15',
 '42',
 'Ġ15',
 '86',
 'Ġ378',
 '1',
 'Ġ24',
 '71',
 'Ġ15',
 '25',
 'Ġ18',
 '59',
 'Ġ16',
 '69',
 'Ġ25',
 '12',
 'Ġ457',
 '2',
 'Ġ15',
 '46',
 'Ġ16',
 '09',
 'Ġ378',
 '1',
 'Ġ24',
 '71',
 'Ġ15',
 '25',
 'Ġ3',
 '39